In [ ]:
#!pip install --upgrade pip
#!pip install --upgrade setuptools
#!pip install GEOS

#!pip install GeobricksProj4ToEPSG
#!conda install -c conda-forge cartopy --yes
#!pip install geopandas
#print('ok')

In [ ]:
import os
import pandas
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import cascaded_union, unary_union#, union
import time
#import rtree

os.chdir('/home/idies/workspace/Temporary/raddick/cra_scratch/')
print(os.getcwd())

In [ ]:
nfile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/Neighborhoods.shp'
ngeo = gpd.read_file(nfile)
print('Geographies read')
#ngeo = ngeo.set_index('Name')
#ngeo = ngeo.assign(percent_white=100*(ngeo['White']/ngeo['Population']))
#ngeo = ngeo.assign(percent_black=100*(ngeo['Blk_AfAm']/ngeo['Population']))
ngeo = ngeo.assign(percent_white=(ngeo['White']/ngeo['Population']))
ngeo = ngeo.assign(percent_black=(ngeo['Blk_AfAm']/ngeo['Population']))

#ngeo.head(1)

ngeo.loc[ngeo['Population'] == 0, 'percent_black'] = np.nan
#ngeo['percent_black'] = np.nan
#print('OK')
ngeo[['Name', 'Population', 'percent_black']].sort_values(by='Population')

In [ ]:
fig, ax = plt.subplots(figsize=(48,48))

#thecmap = plt.cm.Greys
thecmap = plt.cm.viridis_r
thecmap.set_under(color='r', alpha=0.2)
ngeo.plot(column='percent_black', ax=ax, cmap=thecmap, edgecolor='black')

ax.set_aspect('equal')
ax.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
#ax.annotate('ED', xy=(-76.609383, 39.299236), xytext=(-42.5, 35),
#             arrowprops=dict(facecolor='black', shrink=0.05),
#             fontsize=22
#            )

for ix, thisrow in ngeo.iterrows():
    thisrow['Name'] = thisrow['Name'].replace('-', '- ')
    thisrow['Name'] = thisrow['Name'].replace('/', '/ ')
    thisrow['Name'] = thisrow['Name'].replace(' ', '\n')
    annotator = thisrow['Name']
    if (thisrow['Population'] > 0):
        annotator += '\n' + '{0:,.0%}'.format(thisrow['percent_black'])
        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['Population'])
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)
    

# add colorbar
cax1 = fig.add_axes([0.21, 0.08, 0.61, 0.03])
sm1 = plt.cm.ScalarMappable(cmap=thecmap, norm=plt.Normalize(vmin=0, vmax=100))


# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax1.tick_params(labelsize=48)
cbar1.set_label('Percent black', fontsize=64)

plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/segregation_labeled_huge.svg', format='svg')
print('ok')

In [ ]:
ngeo.columns

In [ ]:
cfile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/census2010_neighborhoods.shp'
#dfile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/census2010_neighborhoods.shp'
c2010 = gpd.read_file(cfile)
c2010.columns#head(1)

# Just print neighborhoods with labels, no color scale

In [ ]:
nfile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/Neighborhoods.shp'
ngeo = gpd.read_file(nfile)
ngeo = ngeo.set_index('OBJECTID')

print('Geographies read')

ngeo['percent_black'] = np.nan

fig, ax = plt.subplots(figsize=(48,48))

thecmap = plt.cm.viridis_r
thecmap.set_under(color='w')#, alpha=0.2)
ngeo.plot(column='percent_black', ax=ax, cmap=thecmap, edgecolor='black')

ax.set_aspect('equal')
ax.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

for ix, thisrow in ngeo.iterrows():
    thisrow['Name'] = thisrow['Name'].replace('-', '- ')
    thisrow['Name'] = thisrow['Name'].replace('/', '/ ')
    thisrow['Name'] = thisrow['Name'].replace(' ', '\n')
    annotator = thisrow['Name']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14)
    
#plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/just_neighborhoods_huge.svg', format='svg')
#print('ok')
plt.show()

In [ ]:
#os.getcwd()
ngeo[['Name', 'Population']].sort_values(by='Name').to_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/neighborhoods_list.csv')
print('ok')

In [ ]:
# PLOT CSAs

csafile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/csa/community_statistical_area.shp'
csa_gdf = gpd.read_file(csafile)
csa_gdf = csa_gdf.set_index('NEIG55ID')
csa_gdf['percent_black'] = np.nan
csa_gdf = csa_gdf.set_geometry('geometry')
csa_gdf.sample(1)

fig, ax = plt.subplots(figsize=(48,48))

thecmap = plt.cm.viridis_r
thecmap.set_under(color='w')#, alpha=0.2)
csa_gdf.plot(column='percent_black', ax=ax, cmap=thecmap, edgecolor='black')

ax.set_aspect('equal')
ax.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

for ix, thisrow in csa_gdf[csa_gdf['DC55NEIG'] != 'Tract Not Assigned'].iterrows():
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].replace('-', '- ')
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].replace('/', '/ ')
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].replace(' ', '\n')
    annotator = thisrow['DC55NEIG']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14)
plt.show()

#plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/csa/all_csa.svg', format='svg')
#print('OK')

In [ ]:
# PLOT CSAs

csafile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/csa/community_statistical_area.shp'
csa_gdf = gpd.read_file(csafile)
csa_gdf = csa_gdf.set_index('NEIG55ID')
csa_gdf['percent_black'] = np.nan
csa_gdf = csa_gdf.set_geometry('geometry')
csa_gdf.sample(1)

fig, ax = plt.subplots(figsize=(48,48))

thecmap = plt.cm.viridis_r
thecmap.set_under(color='w', alpha=0)
csa_gdf.plot(column='percent_black', ax=ax, cmap=thecmap, edgecolor='red', linewidth=6)
ngeo.plot(column='percent_black', ax=ax, cmap=thecmap, edgecolor='black')

ax.set_aspect('equal')
ax.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

for ix, thisrow in csa_gdf[csa_gdf['DC55NEIG'] != 'Tract Not Assigned'].iterrows():
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].replace('-', '- ')
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].replace('/', '/ ')
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].replace(' ', '\n')
    thisrow['DC55NEIG'] = thisrow['DC55NEIG'].upper()
    annotator = thisrow['DC55NEIG']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=18, color='red')

for ix, thisrow in ngeo.iterrows():
    thisrow['Name'] = thisrow['Name'].replace('-', '- ')
    thisrow['Name'] = thisrow['Name'].replace('/', '/ ')
    thisrow['Name'] = thisrow['Name'].replace(' ', '\n')
    annotator = thisrow['Name']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14)

#plt.show()

plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/neighborhood_and_csa.svg', format='svg')
print('OK')